# Time Series with Azure Data Explorer

Azure Data Explorer (ADX) is available at [https://dataexplorer.azure.com](https:\dataexplorer.azure.com\), where you will find the main ADX administration portal. The portal allows you to create, clusters, databases, tables and all the objects in ADX, moreover it allows you to run KQL commands to query data and perform administrative tasks.

There are more query tools available, like Kusto Explorer or Azure Data Studio, which can run Kusto notebooks, such as this one.

## **Initial Setup:**

The database already contains a table with 80M rows. Not a lot but enough to play with. Open [https://dataexplorer.azure.com](https:\dataexplorer.azure.com\) to see clusters, databases and tables.

Let's create a smaller copy to demonstrate some concepts.

In [ ]:
// setup: drop and recreate a small table as a copy of the bigger table
.drop table sqlserver_performance_small ifexists;

In [ ]:
// copy table 100 rows from sqlserver_performance to sqlserver_performance_small
.set sqlserver_performance_small 
<| sqlserver_performance | where counter startswith "Page Life Expectancy" | order by ['time'] asc | take 100;

In [ ]:
// append 100 rows from sqlserver_performance to sqlserver_performance_small changing the date to now
.append sqlserver_performance_small <| 
sqlserver_performance 
| order by ['time'] asc 
| take 100 
| project name, ['time'] = now(), agent, company, counter, counter_type, host, instance, measurement_db_type, object, sql_instance, value;

## **How efficient is data compression?**

ADX is pretty efficient at storing data because it uses columnar compression. Other databases can do that, with different results.

How big is our 80M rows table?

In [ ]:
.show table [@"sqlserver_performance"] details 
| project TotalRowCount, HotExtentSize, HotRowCount, TotalOriginalSize

## **Basic KQL Syntax**

The idea behind KQL is that every element of the language is piped to the next one, so you always start from a table and then you pipe it to something else, like a where clause or a project clause.

The simplest possible KQL query simply uses the table name to retrieve the contents of the table: all rows and all columns.

In [ ]:
sqlserver_performance_small

In [ ]:
// if you don't know how to write it KQL, try SQL ==> does not work in Azure Data Studio, only on https://dataexplorer.azure.com
EXPLAIN
SELECT * FROM sqlserver_performance_small WHERE time > '2022-10-05'

In [ ]:
// limit results to 10 rows
sqlserver_performance_small
| take 10

In [ ]:
// limit the columns that you want
sqlserver_performance_small 
| project ['time']       // notice how we had to escape the keyword "time"
        , sql_instance
        , counter
        , object
        , value

In [ ]:
// basic filtering with the where clause
sqlserver_performance_small 
| project ['time'], sql_instance, counter, object, value
| where counter == 'Page Life Expectancy'  // notice the == comparison

// wait, what? no data? it considers trailing spaces...

In [ ]:
// filter with common patterns
sqlserver_performance_small 
| project ['time'], sql_instance, counter, object, value
| where counter startswith "Page Life Expectancy"


In [ ]:
// filter with common patterns
sqlserver_performance_small 
| project ['time'], sql_instance, counter, object, value
| where counter endswith "Expectancy"  // this time trailing spaces are not considered, WTH?

In [ ]:
// filter with common patterns
sqlserver_performance_small 
| project ['time'], sql_instance, counter, object, value
| where counter contains "Life"

In [ ]:
// sorting results
sqlserver_performance_small 
| project ['time'], sql_instance, counter, object, value
| order by sql_instance, counter, object asc // if you omit the sort order it defaults to desc (WHY???!???)

## **Time-Series queries with KQL**

Is ADX a time-series database? Well, not only, so the data in ADX tables is not necessarily time-series data.

In [ ]:
// filter on a specific date/time constant
sqlserver_performance_small
| where ['time'] > todatetime('2022-09-30') 

In [ ]:
// filter on a time interval
sqlserver_performance
| where ['time'] between (todatetime('2022-09-30 10:00:00') .. todatetime('2022-09-30 10:01:00'))

In [ ]:
// use shorthands for well known time intervals
sqlserver_performance_small
| where ['time'] < now()

In [ ]:
// use shorthands for well known time intervals
sqlserver_performance_small
| where ['time'] > now(-1d) // data since yesterday

In [ ]:
// data from last week
sqlserver_performance_small
| where ['time'] > now(-7d) and ['time'] < now()

## **Using `make-series` to create time-series data**

OK, but this is not real time-series data: it is tabular data. Time-series is a different beast and in order to have it we can use the `make-series` operator. The columns are turned into arrays of data, each element relative to the corresponding element of the time array.

In [ ]:
// this creates a proper time-series result set, with array of calculate values 
// and corresponding array of time intervals
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count() on ['time'] step 1h


In [ ]:
// you can filter the data on a particular interval
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count() on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-15') step 1h

In [ ]:
// you can calculate multiple aggregates in the time intervals
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count(), avg_value = avg(value) 
        on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-15') step 1h

In [ ]:
// the columns that you don`t aggregate can be used to group the data
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count(), avg_value = avg(value) 
        on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-15') step 1h
        by sql_instance, object

In [ ]:
// to get back to a tabular format for the results, you can use mv-expand
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count(), avg_value = avg(value) 
        on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-15') step 1h
        by sql_instance, object
| mv-expand ['time'], num_rows, avg_value

In [ ]:
// you can change the output to a chart with the render command
// only works on the web, does not work in ADS
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series num_rows = count(), avg_value = avg(value) 
        on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-13') step 1h
| render timechart // ignored by ADS


In [ ]:
// if you look at the results, you will see that the missing
// values are filled automatically when calculating the aggregations
// use default to control this behavior
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series 
        num_rows = count(), 
        avg_value = avg(value) default = double(null) // what happens when a value is missing? default controls it
        on ['time'] from todatetime('2022-10-06') to todatetime('2022-10-07') step 1h
| render timechart


In [ ]:
// what if a constant value is not enough?
// use series_fill_* to fill the blanks
sqlserver_performance
| where counter startswith('Page Life Expectancy')
| make-series 
        num_rows = count(), 
        avg_value = avg(value) default = double(null)
        on ['time'] from todatetime('2022-10-06') to todatetime('2022-10-07') step 1h
| extend series_fill_backward(avg_value)
| render timechart


In [ ]:
// you can perform intersting transformations to the data, like plotting a trend line
sqlserver_performance
| where  counter in ('Data File(s) Size (KB)')
| make-series spaceused = max(value) default = long(null) on ['time'] from todatetime('2022-09-30') to todatetime('2022-10-15') step 1d
| extend spaceused = series_fill_forward(series_fill_backward(spaceused))
| extend (rsquare, slope, variance, rvariance, interception, line_fit) = series_fit_line(spaceused)
| project ['time'], spaceused, line_fit
| render timechart with (xcolumn=['time'], ycolumns=spaceused, line_fit)


## **Retention Policies and Downsampling**

Data is not useful forever. Usually you want to keep troubleshooting data for a short period of time (7 days? 30 days?) and then delete it or downsample it.

SoftDeletePeriod controls how long the data is guaranteed to be available after ingestion. After that, data is soft deleted.

Recoverability enabled/disabled controls whether you can undelete soft-deleted data for a period of 14 days after deletion.

In [ ]:
// set retention policy for entire database
.alter database ADXdemo policy retention "{'SoftDeletePeriod': '365000.00:00:00', 'Recoverability':'Enabled'}"

In [ ]:
// set retention policy for single table
.alter-merge table sqlserver_performance_small policy retention softdelete = 7d recoverability = disabled;

**Downsampling** is the operation that involves aggregating data to avoid keeping high-frequency data for too long, in order to save space.

Of course in this case we have one data point every 15 seconds, so this is  not high-frequency data at all. However, we could decide to delete the original table after let's say 30 days and keep downsampled data for longer, like 1 year.

How do you schedule that? Power Automate is an option.

In [ ]:
// drop any pre-existing data
.drop table sqlserver_performance_10m ifexists;

In [ ]:
// downsample the data and populate the target table
.set-or-append async sqlserver_performance_10m <| 
sqlserver_performance
| make-series 
        avg_value = avg(value) default = double(null),
        min_value = min(value) default = double(null),
        max_value = min(value) default = double(null)
        on ['time'] //from ago(7d) to now() 
        step 10m 
        by agent, company, counter, counter_type, host, instance, measurement_db_type, name, object, sql_instance
| mv-expand ['time'], avg_value, min_value, max_value
| where isnotnull(avg_value);

In [ ]:
sqlserver_performance_10m | where counter startswith 'Page Life Expectancy' | take 1000;

In [ ]:
sqlserver_performance_10m | count